# Catastrophe

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### Connection à la base

In [ ]:
# Define Neo4j connections
import pandas as pd
import random
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

# Configuration de l'option d'affichage
pd.set_option('display.max_rows', None)

In [ ]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

#### Suppression des données

In [ ]:
delete_node = """
    MATCH (n)
    WHERE NOT labels(n) = ['_GraphConfig']
    DETACH DELETE n
"""
run_query(delete_node)

#### Configuration de la base

In [ ]:
deleConfig = """
CALL n10s.graphconfig.drop
"""
run_query(deleConfig)

n10s_config_query = """
CALL n10s.graphconfig.init({ handleVocabUris: "IGNORE", keepLangTag: false, handleMultival: "OVERWRITE"});
"""
run_query(n10s_config_query)

In [ ]:
n10s_config_query = """
CREATE CONSTRAINT  n10s_unique_uri IF NOT EXISTS FOR (r:Resource) REQUIRE r.uri IS UNIQUE;
"""
run_query(n10s_config_query)

#### Importaion de l'ontologie

In [ ]:
import_ontology = """
    CALL n10s.onto.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/file_11.ttl", "Turtle") 
"""


run_query(import_ontology)

#### Nombre de secouristes

In [ ]:
rsecouriste = """
CALL n10s.rdf.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/file6.ttl","Turtle");
"""
run_query(rsecouriste)
labels = ["Médecin", "Militaire", "Infirmier", "Pompiers"]
for label in labels:
    query2 = f'''
                MATCH (n:{label})
                UNWIND n.NombreDindividus AS nombre 
                MERGE (n)-[:NombreDindividus]->(:Effectif {{value: nombre}})'''
    run_query(query2)
    query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
    run_query(query)

| Catastrophe  | QCode         | 
| :--------------- |:---------------:| 
|Séisme | Q7944 |
|Inondation | Q76337338|
|IncendieDeForet | Q169950|
|Tempete | Q81054|
|Secheresse | Q43059|
|GlissementDeTerrain | Q167903 |
|Tsunami | Q8070|
|cyclone | Q79602|
|AccidentDeTrain | Q1078765|
|IncendieDeMaison | Q7625093|
|Explosion | Q179057|
|AccidentNucleaire | Q1620824|

   

In [ ]:
disasters = {
    "Seisme": "Q7944",
    "Inondation": "Q8068",
    "IncendieDeForet": "Q169950",
    "Tempete": "Q81054",
    "Secheresse": "Q43059",
    "Tsunami": "Q8070",
    "AccidentDeTrain": "Q1078765",
    "IncendieDeMaison": "Q7625093",
    "Explosion": "Q179057"
    # "AccidentNucleaire": "Q1620824",
    # "GlissementDeTerrain": "Q167903",
    # "Cyclone": "Q79602"
}

In [ ]:
def get_random_disaster():
    liste_de_tuples = list(disasters.items())
    tuple_aléatoire = random.choice(liste_de_tuples)
    clé_aléatoire = tuple_aléatoire[0]
    valeur_aléatoire = tuple_aléatoire[1]
    return clé_aléatoire, valeur_aléatoire

#### Ajout d'une catastrophe

In [ ]:
clé_aléatoire, valeur_aléatoire = get_random_disaster()

query = f'''
WITH 
'
CONSTRUCT {{
  ?event a schema:{clé_aléatoire}, schema:Event ;
          schema:name ?eventLabel ;
          schema:description ?description ;
          schema:startDate ?date ;
          schema:aLieu ?location ;
          schema:numberOfInjured ?numberOfInjured ;
          schema:numberOfDeaths ?numberOfDeaths ;
          schema:aLieu ?locatedIn .
  ?location rdfs:label ?locationLabel ; a schema:Country, schema:SpatialThing.
  ?locatedIn rdfs:label ?locatedInLabel ; a schema:City, schema:SpatialThing.
}} WHERE {{
  ?event wdt:P31/wdt:P279* wd:{valeur_aléatoire} ;
          wdt:P17 ?location ;
          wdt:P585 ?date .
  OPTIONAL {{ ?event wdt:P2320 ?aftershocks . }}
  OPTIONAL {{ ?event wdt:P18 ?image . }}
  OPTIONAL {{ ?event wdt:P10 ?video . }}
  OPTIONAL {{ ?event wdt:P2528 ?magnitude . }}
  OPTIONAL {{ ?event wdt:P1339 ?numberOfInjured . }}
  OPTIONAL {{ ?event wdt:P1120 ?numberOfDeaths . }}
  OPTIONAL {{ ?event wdt:P131 ?locatedIn . ?locatedIn rdfs:label ?locatedInLabel . FILTER(LANG(?locatedInLabel) = "fr") }}
  OPTIONAL {{ ?event schema:description ?description . FILTER(LANG(?description) = "fr") }}
  ?location rdfs:label ?locationLabel . FILTER(LANG(?locationLabel) = "fr")
  ?event rdfs:label ?eventLabel . FILTER(LANG(?eventLabel) = "fr")
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }}
}} ORDER BY RAND()
LIMIT 1
' AS sparql
CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" + 
    apoc.text.urlencode(sparql),"JSON-LD", 
  {{ headerParams: {{ Accept: "application/ld+json"}} , 
    handleVocabUris: "IGNORE"}}) 
YIELD terminationStatus, triplesLoaded 
RETURN terminationStatus, triplesLoaded
'''

print(clé_aléatoire)

run_query(query)

unwind = f'''
MATCH (n:{clé_aléatoire})
UNWIND n.startDate AS startDate
MERGE (n)-[:aPourDate]->(:Date {{ value: startDate }})

WITH n
UNWIND n.numberOfDeaths AS numberOfDeaths
MERGE (n)-[:Implique]->(:Morts {{ value: numberOfDeaths }})

WITH n
UNWIND n.numberOfInjured AS numberOfInjured
MERGE (n)-[:Implique]->(:Blesses {{ value: numberOfInjured }})
'''
run_query(unwind)

labels = ["Seisme", "Inondation", "IncendieDeForet", "Tempete", "Secheresse", "GlissementDeTerrain", "Tsunami", "Cyclone", "Date", "SpatialThing", 
          "AccidentDeTrain", "IncendieDeMaison", "Explosion", "AccidentNucleaire", "Morts", "Blesses"]
for label in labels:
    query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
    run_query(query)

In [ ]:
injuryOrDeath = """
    MATCH (injury)<-[:Implique]-(event:Event)-[:Implique]->(death)
    WHERE (death:Morts) OR (injury:Blesses)
    RETURN event.name as Catastrophe, injury.value as Blessés, id(injury) as idBlessés, death.value as Morts, id(death) as idMorts
"""
res = run_query(injuryOrDeath)
res

In [ ]:
if res["Blessés"].eq(0).all():
    print("La valeur de 'Blessés' est égale à zéro.")
else:
    print("La valeur de 'Blessés' n'est pas égale à zéro.")
    id_blesses = res["idBlessés"]
    with driver.session() as session:

        session.run("""CREATE (equipe:EquipeDeSecours) SET equipe.name = "Equipe"
            CREATE (soigner:Soigner) SET soigner.name = "Soigner"
            CREATE (equipe)-[:Effectue]->(soigner)
            WITH soigner, equipe
            MATCH (b:Blesses)
            WHERE id(b) = $idBlesses
            CREATE (soigner)-[:OperationDeSoins]->(blesses)""", idBlesses=id_blesses)


    labels = ["EquipeDeSecours", "Soigner"]
    for label in labels:
        query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
        run_query(query)


In [ ]:
# unwind = f'''
# MATCH (n:{clé_aléatoire})
# UNWIND n.startDate AS startDate
# MERGE (n)-[:aPourDate]->(:Date {{ value: startDate }})

# WITH n
# UNWIND n.numberOfDeaths AS numberOfDeaths
# MERGE (n)-[:Implique]->(:Morts {{ value: numberOfDeaths }})

# WITH n
# UNWIND n.numberOfInjured AS numberOfInjured
# MERGE (n)-[:Implique]->(:Blesses {{ value: numberOfInjured }})
# '''
# run_query(unwind)

In [ ]:
# labels = ["Seisme", "Inondation", "IncendieDeForet", "Tempete", "Secheresse", "GlissementDeTerrain", "Tsunami", "Cyclone", "Date", "SpatialThing", 
#           "AccidentDeTrain", "IncendieDeMaison", "Explosion", "AccidentNucleaire", "Morts", "Blesses"]
# for label in labels:
#     query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
#     run_query(query)

| Type de secour  | Nombre de victimes    | Nombre de secours | 
| :--------------- |:---------------:| :---------------:| 
|Séisme | Q7944 | Q7944 |

